In [90]:
import pandas as pd
import numpy as np
import html
import string
from sklearn.model_selection import train_test_split

np.random.seed(42)

In [91]:
def preprocess_titles(df):
    return (
        df.title.str.lower() # normalize to lower case
                .str.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))) # replace punctuation characters with spaces
                .map(lambda x: ' '.join(x.split())) # remove double spaces
    )

def preprocess_authors(df):
    return (
         df.loc[df.authors.notna(), 'authors'] # select only rows with some authors
            .map(lambda x: html.unescape(x)) # convert html escaped latin characters
            .map(lambda x: ', '.join(sorted(x.split(', ')))) # sort authors alphabetically
    )

In [92]:
df_true = pd.read_csv('DBLP-ACM/DBLP-ACM_perfectMapping.csv')
df_true

,idDBLP,idACM
0,conf/sigmod/SlivinskasJS01,375678
1,conf/sigmod/ChaudhuriDN01,375694
2,conf/sigmod/RinfretOO01,375669
3,conf/sigmod/BreunigKKS01,375672
4,conf/sigmod/JagadishJOT01,375687
...,...,...
2219,journals/sigmod/Scholl01,604275
2220,journals/sigmod/Rosneblatt94,190649
2221,journals/sigmod/Winslett02b,601871
2222,journals/sigmod/Labrinidis01,604283


In [93]:
dfa = pd.read_csv('DBLP-ACM/ACM.csv')
dfa.loc[:, 'title'] = preprocess_titles(dfa)
dfa.loc[dfa.authors.notna(), 'authors'] = preprocess_authors(dfa)
dfa

,id,title,authors,venue,year
0,304586,the wasa2 object oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,a user centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,world wide database integrating the web corba ...,"Athman Bouguettaya, Boualem Benatallah, James ...",International Conference on Management of Data,1999
3,304590,xml based information mediation with mix,"Amarnath Gupta, Bertram Ludäscher, Chaitan Bar...",International Conference on Management of Data,1999
4,304582,the ccube constraint object oriented database ...,"Alexander Brodsky, Jia Chen, Paval A. Exarkhop...",International Conference on Management of Data,1999
...,...,...,...,...,...
2289,672977,dual buffering strategies in object bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
2290,950482,guest editorial,"Philip A. Bernstein, Raghu Ramakrishnan, Yanni...",The VLDB Journal &mdash; The International Jou...,2003
2291,672980,graphdb modeling and querying graphs in databases,Ralf Hartmut Güting,Very Large Data Bases,1994
2292,945741,review of the data warehouse toolkit the compl...,Alexander A. Anisimov,ACM SIGMOD Record,2003


In [94]:
dfb = pd.read_csv('DBLP-ACM/DBLP2.csv', encoding='latin_1')
dfb.loc[:, 'title'] = preprocess_titles(dfb)
dfb.loc[dfb.authors.notna(), 'authors'] = preprocess_authors(dfb)
dfb

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,semantic integration of environmental models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,estimation of query result distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,incremental maintenance for non distributive a...,"Hamid Pirahesh, Richard Sidle, Roberta Cochran...",VLDB,2002
3,conf/vldb/GardarinGT96,cost based selection of path expression proces...,"Georges Gardarin, Jean-Robert Gruser, Zhao-Hui...",VLDB,1996
4,conf/vldb/HoelS95,benchmarking spatial join operations with spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...,...
2611,journals/tods/KarpSP03,a simple algorithm for finding frequent elemen...,"Christos H. Papadimitriou, Richard M. Karp, Sc...",ACM Trans. Database Syst.,2003
2612,conf/vldb/LimWV03,sash a self adaptive histogram set for dynamic...,"Jeffrey Scott Vitter, Lipyeow Lim, Min Wang",VLDB,2003
2613,journals/tods/ChakrabartiKMP02,locally adaptive dimensionality reduction for ...,"Eamonn J. Keogh, Kaushik Chakrabarti, Michael ...",ACM Trans. Database Syst.,2002
2614,journals/sigmod/Snodgrass01,chair s message,Richard T. Snodgrass,SIGMOD Record,2001


In [95]:
def merge_dataframes(dfa, dfb, df_true, match_number_of_false=False):
    df_match = (
        dfa.merge(df_true, left_on="id", right_on="idACM")
           .merge(dfb, left_on="idDBLP", right_on="id")
    ) # take only matches in the two data frames
    df_match["label"] = True # mark these rows as matches

    dfa_no_match = df_match[["id_x"]].merge(dfa, how="right", left_on="id_x", right_on="id") # join with the matches
    dfa_no_match.drop(dfa_no_match[~dfa_no_match["id_x"].isna()].index, inplace=True) # drop all rows, which have a match
    dfa_no_match.drop(["id_x"], axis=1, inplace=True) # drop the column used for join
    dfa_no_match["order"] = np.arange(len(dfa_no_match)) # create new column to join with 'dfb'

    dfb_no_match = df_match[["id_y"]].merge(dfb, how="right", left_on="id_y", right_on="id") # join with the matches
    dfb_no_match.drop(dfb_no_match[~dfb_no_match["id_y"].isna()].index, inplace=True) # drop all rows, which have a match
    dfb_no_match.drop(["id_y"], axis=1, inplace=True) # drop the column used for join
    dfb_no_match["order"] = np.arange(len(dfb_no_match)) # create new column to join with 'dfa'

    df_no_match = dfa_no_match.merge(dfb_no_match, on="order") # join non matches with inner join, some records from longer data frame are ignored 
    df_no_match.drop(["order"], axis=1, inplace=True) # drop the column used for join
    df_no_match["label"] = False # mark these rows as not matching

    df = pd.concat([df_match, df_no_match]) # append matches and non matches
    df.drop(["idDBLP", "idACM", "id_x", "id_y"], axis=1, inplace=True) # drop unnecessary columns

    if match_number_of_false: # add non matches to the data frame so there are 50/50 matches/non matches
        add_no_match_count = len(df[df["label"] == True]) - len(df[df["label"] == False])

        if add_no_match_count > 0: # there are more matches then non matches
            dfa_with_connection = dfa.merge(df_true, left_on="id", right_on="idACM") # add the connection table, so matches/non matches can be found
            dfa_with_connection["order"] = np.random.choice(len(dfa_with_connection), len(dfa_with_connection), replace=False) # random order to join randomly
            dfb_copy = dfb.copy() # copy so the function doesn't have side effects
            dfb_copy["order"] = np.arange(len(dfb_copy)) # column to join with 'dfa_with_connection'
            df_no_match = dfa_with_connection.merge(dfb_copy, left_on="order", right_on="order") # join on the order
            df_no_match[df_no_match["idACM"] != df_no_match["id_y"]] # remove matches
            df_no_match = df_no_match[:add_no_match_count] # take up to the needed number of non matches
            df_no_match["label"] = False # mark them as non matches
            df_no_match.drop(["idDBLP", "idACM", "id_x", "id_y", "order"], axis=1, inplace=True) # drop unnecessary columns 
            df = pd.concat([df, df_no_match]) # add them to the final data frame

    return df.sample(frac=1) # shuffel rows of the data frame

In [96]:
NUM_OF_CROSS_VALIDATIONS = 5

dfa_train, dfa_test = train_test_split(dfa, test_size=1/(NUM_OF_CROSS_VALIDATIONS + 1), stratify=dfa[["year"]], random_state=42)
dfb_train, dfb_test = train_test_split(dfb, test_size=1/(NUM_OF_CROSS_VALIDATIONS + 1), stratify=dfb[["year"]], random_state=42)

df_train_validation = merge_dataframes(dfa_train, dfb_train, df_true, True) # ensure there is same number of positive and negative training samples
df_test = merge_dataframes(dfa_test, dfb_test, df_true)

y_test = df_test.pop("label")
X_test = df_test

train_validation_data = []
for _ in range(NUM_OF_CROSS_VALIDATIONS):
    data_dict = {}
    df_train, df_validation = train_test_split(df_train_validation, test_size=1/NUM_OF_CROSS_VALIDATIONS, stratify=df_train_validation[["label"]])
    data_dict["y_train"] = df_train.pop("label")
    data_dict["y_validation"] = df_validation.pop("label")
    data_dict["X_train"] = df_train
    data_dict["X_validation"] = df_validation
    train_validation_data.append(data_dict)

In [97]:
train_validation_data[0]["X_train"]

,title_x,authors_x,venue_x,year_x,title_y,authors_y,venue_y,year_y
709,interviewing during a tight job market,"Qiong Luo, Zachary G. Ives",ACM SIGMOD Record,2002,xml and information retrieval a sigir 2000 wor...,"Aya Soffer, David Carmel, Yoëlle S. Maarek",SIGMOD Record,2001
509,space optimization in deductive databases,"Divesh Srivastava, Jeffrey F. Naughton, Raghu ...",ACM Transactions on Database Systems (TODS),1995,space optimization in deductive databases,"Divesh Srivastava, Jeffrey F. Naughton, Raghu ...",ACM Trans. Database Syst.,1995
727,towards on line analytical mining in large dat...,Jiawei Han,ACM SIGMOD Record,1998,parallel query scheduling and optimization wit...,"Minos N. Garofalakis, Yannis E. Ioannidis",VLDB,1997
426,efficient evaluation of queries in a mediator ...,"Laura Bright, Louiqa Raschid, Maria Esther Vid...",International Conference on Management of Data,2002,providing dynamic security control in a federa...,"N. B. Idris, R. F. Churchhouse, W. A. Gray",VLDB,1994
1417,temporal aggregation in active database rules,"Carlo Zaniolo, Iakovos Motakis",International Conference on Management of Data,1997,temporal aggregation in active database rules,"Carlo Zaniolo, Iakovos Motakis",SIGMOD Conference,1997
...,...,...,...,...,...,...,...,...
238,describing semistructured data,Luca Cardelli,ACM SIGMOD Record,2001,data management for pervasive computing,"Michael J. Franklin, Mitch Cherniack, Stanley ...",VLDB,2001
19,a general technique for querying xml documents...,"Efstratios Viglas, Eugene Shekita, Igor Tatari...",ACM SIGMOD Record,2001,a database platform for bioinformatics,Sandeepan Banerjee,VLDB,2000
838,efficiently publishing relational data as xml ...,"Berthold Reinwald, Bruce G. Lindsay, Eugene J....",Very Large Data Bases,2000,efficiently publishing relational data as xml ...,"Berthold Reinwald, Bruce G. Lindsay, Eugene J....",VLDB,2000
266,approximate join processing over data streams,"Abhinandan Das, Johannes Gehrke, Mirek Riedewald",International Conference on Management of Data,2003,approximate join processing over data streams,"Abhinandan Das, Johannes Gehrke, Mirek Riedewald",SIGMOD Conference,2003


In [98]:
train_validation_data[0]["y_train"]

709     False
509      True
727     False
426     False
1417     True
        ...  
238     False
19      False
838      True
266      True
1205     True
Name: label, Length: 2478, dtype: bool

In [99]:
train_validation_data[0]["X_validation"]

,title_x,authors_x,venue_x,year_x,title_y,authors_y,venue_y,year_y
902,the third manifesto,"C. J. Date, Hugh Darwen",ACM SIGMOD Record,1995,the third manifesto,"C. J. Date, Hugh Darwen",SIGMOD Record,1995
585,distributed hypertext resource discovery throu...,"Byron Dom, Martin van den Berg, Soumen Chakrab...",Very Large Data Bases,1999,distributed hypertext resource discovery throu...,"Byron Dom, Martin van den Berg, Soumen Chakrab...",VLDB,1999
68,next generation database systems won t work wi...,John Mylopoulos,International Conference on Management of Data,1998,improving business process quality through exc...,"Daniela Grigori, Fabio Casati, Ming-Chien Shan...",VLDB,2001
873,i3 intelligent interactive investigation of ol...,"Gayatri Sathe, Sunita Sarawagi",International Conference on Management of Data,2000,an ontological analysis of the relationship co...,"Ron Weber, Veda C. Storey, Yair Wand",ACM Trans. Database Syst.,1999
2,curio a novel solution for efficient storage a...,"Anindya Datta, Helen M. Thomas, Krithi Ramamri...",Very Large Data Bases,1999,topaz a cost based rule driven multi phase par...,"Bernhard Mitschang, Clara Nippl",VLDB,1998
...,...,...,...,...,...,...,...,...
1432,performance issues in incremental warehouse ma...,"Hector Garcia-Molina, Jennifer Widom, Jun Yang...",Very Large Data Bases,2000,performance issues in incremental warehouse ma...,"Hector Garcia-Molina, Jennifer Widom, Jun Yang...",VLDB,2000
688,a critique of ansi sql isolation levels,"Elizabeth O'Neil, Hal Berenson, Jim Gray, Jim ...",International Conference on Management of Data,1995,are quorums an alternative for data replication,"Bettina Kemme, Gustavo Alonso, Marta Patiño-Ma...",ACM Trans. Database Syst.,2003
331,local dimensionality reduction a new approach ...,"Kaushik Chakrabarti, Sharad Mehrotra",Very Large Data Bases,2000,local dimensionality reduction a new approach ...,"Kaushik Chakrabarti, Sharad Mehrotra",VLDB,2000
1072,oodb indexing by class division,"Paris C. Kanellakis, Sridhar Ramaswamy",International Conference on Management of Data,1995,a foundation for multi dimensional databases,"Laks V. S. Lakshmanan, Marc Gyssens",VLDB,1997


In [100]:
train_validation_data[0]["y_validation"]

902      True
585      True
68      False
873     False
2       False
        ...  
1432     True
688     False
331      True
1072    False
1180     True
Name: label, Length: 620, dtype: bool